In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]

load gdsc2


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for target_index in tqdm(np.arange(res.shape[dim])):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = nihgcn_new(
                cell_exprs=exprs,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/240 [00:00<?, ?it/s]

epoch:   0 loss:0.709002 auc:0.4520
epoch:  20 loss:0.153209 auc:0.7782
epoch:  40 loss:0.136905 auc:0.7480
epoch:  60 loss:0.123181 auc:0.7450
epoch:  80 loss:0.114529 auc:0.7411
epoch: 100 loss:0.108614 auc:0.7862
epoch: 120 loss:0.102904 auc:0.8392
epoch: 140 loss:0.097554 auc:0.8728
epoch: 160 loss:0.093757 auc:0.8831
epoch: 180 loss:0.089240 auc:0.8947
epoch: 200 loss:0.088499 auc:0.8843
epoch: 220 loss:0.083874 auc:0.9143
epoch: 240 loss:0.081974 auc:0.9151
epoch: 260 loss:0.079845 auc:0.9091
epoch: 280 loss:0.079814 auc:0.9015
epoch: 300 loss:0.077965 auc:0.9287
epoch: 320 loss:0.076641 auc:0.9257
epoch: 340 loss:0.076715 auc:0.9249
epoch: 360 loss:0.076369 auc:0.9216
epoch: 380 loss:0.075000 auc:0.9233
epoch: 400 loss:0.074535 auc:0.9273
epoch: 420 loss:0.074820 auc:0.9299
epoch: 440 loss:0.073995 auc:0.9334
epoch: 460 loss:0.074004 auc:0.9199
epoch: 480 loss:0.073494 auc:0.9366
epoch: 500 loss:0.074390 auc:0.9322
epoch: 520 loss:0.074599 auc:0.9098
epoch: 540 loss:0.073108 auc

  2%|▏         | 4/240 [00:26<26:08,  6.64s/it]

Fit finished.
epoch:   0 loss:0.698697 auc:0.4932
epoch:  20 loss:0.153377 auc:0.7558
epoch:  40 loss:0.137354 auc:0.8113
epoch:  60 loss:0.123877 auc:0.8148
epoch:  80 loss:0.115146 auc:0.8062
epoch: 100 loss:0.109181 auc:0.7927
epoch: 120 loss:0.103416 auc:0.7716
epoch: 140 loss:0.098962 auc:0.7643
epoch: 160 loss:0.093636 auc:0.7479
epoch: 180 loss:0.091060 auc:0.6975
epoch: 200 loss:0.086107 auc:0.7798
epoch: 220 loss:0.084773 auc:0.8030
epoch: 240 loss:0.081823 auc:0.7354
epoch: 260 loss:0.080202 auc:0.7794
epoch: 280 loss:0.078778 auc:0.7577
epoch: 300 loss:0.077666 auc:0.7669
epoch: 320 loss:0.077196 auc:0.7720
epoch: 340 loss:0.076178 auc:0.7720
epoch: 360 loss:0.075723 auc:0.7613
epoch: 380 loss:0.075019 auc:0.7896
epoch: 400 loss:0.074900 auc:0.7876
epoch: 420 loss:0.074671 auc:0.8020
epoch: 440 loss:0.074495 auc:0.7952
epoch: 460 loss:0.076313 auc:0.8188
epoch: 480 loss:0.073850 auc:0.7855
epoch: 500 loss:0.073621 auc:0.7843
epoch: 520 loss:0.075731 auc:0.7240
epoch: 540 los

  5%|▍         | 11/240 [00:43<13:55,  3.65s/it]

Fit finished.
epoch:   0 loss:0.696007 auc:0.4439
epoch:  20 loss:0.153072 auc:0.7483
epoch:  40 loss:0.137128 auc:0.7971
epoch:  60 loss:0.123688 auc:0.8679
epoch:  80 loss:0.115158 auc:0.9161
epoch: 100 loss:0.109111 auc:0.9325
epoch: 120 loss:0.103064 auc:0.9189
epoch: 140 loss:0.097247 auc:0.8912
epoch: 160 loss:0.092599 auc:0.8713
epoch: 180 loss:0.088466 auc:0.8759
epoch: 200 loss:0.085378 auc:0.8787
epoch: 220 loss:0.083088 auc:0.8702
epoch: 240 loss:0.083404 auc:0.8685
epoch: 260 loss:0.080147 auc:0.8673
epoch: 280 loss:0.078351 auc:0.8787
epoch: 300 loss:0.079286 auc:0.8923
epoch: 320 loss:0.077195 auc:0.9082
epoch: 340 loss:0.076209 auc:0.8889
epoch: 360 loss:0.075608 auc:0.8957
epoch: 380 loss:0.075277 auc:0.8985
epoch: 400 loss:0.074761 auc:0.8968
epoch: 420 loss:0.074311 auc:0.8980
epoch: 440 loss:0.074994 auc:0.8929
epoch: 460 loss:0.073950 auc:0.8957
epoch: 480 loss:0.073949 auc:0.8963
epoch: 500 loss:0.073621 auc:0.9019
epoch: 520 loss:0.074145 auc:0.9070
epoch: 540 los

In [ ]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")